In [6]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Anisotropic Diffusion Filter function
def anisodiff_color(img, niter=1, kappa=50, gamma=0.1, step=(1., 1.), option=1):
    img = img.astype('float32')
    imgout = img.copy()
    deltaS = np.zeros_like(imgout)
    deltaE = deltaS.copy()
    NS = deltaS.copy()
    EW = deltaS.copy()
    gS = np.ones_like(imgout)
    gE = gS.copy()

    for ii in range(niter):
        deltaS[:-1, :] = np.diff(imgout, axis=0)
        deltaE[:, :-1] = np.diff(imgout, axis=1)
        if option == 1:
            gS = np.exp(-(deltaS/kappa)**2.)/step[0]
            gE = np.exp(-(deltaE/kappa)**2.)/step[1]
        elif option == 2:
            gS = 1./(1.+(deltaS/kappa)**2.)/step[0]
            gE = 1./(1.+(deltaE/kappa)**2.)/step[1]
        E = gE * deltaE
        S = gS * deltaS
        NS[:] = S
        EW[:] = E
        NS[1:, :] -= S[:-1, :]
        EW[:, 1:] -= E[:, :-1]
        imgout += gamma * (NS + EW)

    return imgout

# Reinhard Stain Normalizer
def standardize_brightness(I):
    p = np.percentile(I, 90)
    return np.clip(I * 255.0 / p, 0, 255).astype(np.uint8)

def lab_split(I):
    I = cv.cvtColor(I, cv.COLOR_RGB2LAB)
    I = I.astype(np.float32)
    I1, I2, I3 = cv.split(I)
    I1 /= 2.55
    I2 -= 128.0
    I3 -= 128.0
    return I1, I2, I3

def merge_back(I1, I2, I3):
    I1 *= 2.55
    I2 += 128.0
    I3 += 128.0
    I = np.clip(cv.merge((I1, I2, I3)), 0, 255).astype(np.uint8)
    return cv.cvtColor(I, cv.COLOR_LAB2RGB)

def get_mean_std(I):
    I1, I2, I3 = lab_split(I)
    m1, sd1 = cv.meanStdDev(I1)
    m2, sd2 = cv.meanStdDev(I2)
    m3, sd3 = cv.meanStdDev(I3)
    means = m1, m2, m3
    stds = sd1, sd2, sd3
    return means, stds

class Normalizer:
    def __init__(self):
        self.target_means = None
        self.target_stds = None

    def fit(self, target):
        target = standardize_brightness(target)
        means, stds = get_mean_std(target)
        self.target_means = means
        self.target_stds = stds

    def transform(self, I):
        I = standardize_brightness(I)
        I1, I2, I3 = lab_split(I)
        means, stds = get_mean_std(I)
        norm1 = ((I1 - means[0]) * (self.target_stds[0] / stds[0])) + self.target_means[0]
        norm2 = ((I2 - means[1]) * (self.target_stds[1] / stds[1])) + self.target_means[1]
        norm3 = ((I3 - means[2]) * (self.target_stds[2] / stds[2])) + self.target_means[2]
        return merge_back(norm1, norm2, norm3)

# Directories setup
input_dir = './demo_datasets_week8'  # Your input dataset directory
output_dir = './Demo_Dataset(anistropic+reinhard)'  # Directory for processed images

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize Normalizer with a target image for stain normalization
target_image_path = './target_image.jpg'  # Replace with your actual target image path
target_image = cv.imread(target_image_path)

# Check if the target image loaded successfully
if target_image is None:
    raise ValueError(f"Failed to load target image from {target_image_path}. Check the path and file format.")

target_image = cv.cvtColor(target_image, cv.COLOR_BGR2RGB)

# Fit normalizer with the target image
normalizer = Normalizer()
normalizer.fit(target_image)

# Define supported image extensions including .tif and .tiff
supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')

# Process each class folder and each image
for class_folder in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_folder)
    if not os.path.isdir(class_path):
        continue
    
    # Create corresponding class folder in the output directory
    output_class_path = os.path.join(output_dir, class_folder)
    if not os.path.exists(output_class_path):
        os.makedirs(output_class_path)
    
    # Process each image in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        
        # Check if the file is an image by extension
        if not image_name.lower().endswith(supported_extensions):
            print(f"Skipping non-image file: {image_path}")
            continue
        
        # Load the image
        image = cv.imread(image_path, cv.IMREAD_UNCHANGED)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue
        
        # Convert to RGB if not already in correct format
        if len(image.shape) == 2 or image.shape[2] == 1:  # Handle grayscale images
            image = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:  # Handle images with alpha channel
            image = cv.cvtColor(image, cv.COLOR_BGRA2RGB)
        else:
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        # Apply Anisotropic Diffusion
        diffused_image = anisodiff_color(image, niter=10, kappa=100, gamma=0.1, step=(1., 1.), option=1)
        
        # Apply Reinhard Stain Normalization
        normalized_image = normalizer.transform(diffused_image)
        
        # Convert back to BGR for saving
        normalized_image_bgr = cv.cvtColor(normalized_image, cv.COLOR_RGB2BGR)
        
        # Save the processed image
        output_image_path = os.path.join(output_class_path, image_name)
        cv.imwrite(output_image_path, normalized_image_bgr)

        print(f"Processed and saved: {output_image_path}")


Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\1015_CRC-Prim-HE-10_020.tif_Row_601_Col_601.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\1040C_CRC-Prim-HE-05_032.tif_Row_1051_Col_2101.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\1050A_CRC-Prim-HE-03_012.tif_Row_301_Col_1351.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\109EF_CRC-Prim-HE-03_012.tif_Row_3151_Col_901.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\10C21_CRC-Prim-HE-05_032.tif_Row_1201_Col_3901.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\110E3_CRC-Prim-HE-07.tif_Row_301_Col_3901.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\11472_CRC-Prim-HE-07.tif_Row_1201_Col_2701.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\1157_CRC-Prim-HE-05_032.tif_Row_1051_Col_2701.tif
Processed and saved: ./Demo_Dataset(anistropic+reinhard)\ADI\11744_CRC-Prim-HE-07.tif_Row_1_Col_3301.tif
Processed and 

In [8]:
import os

# Define the path to the processed dataset directory
output_dir = './Demo_Dataset(anistropic+reinhard)'

# Initialize a dictionary to hold the counts for each class
class_counts = {}

# Iterate through each class folder in the output directory
for class_folder in os.listdir(output_dir):
    class_path = os.path.join(output_dir, class_folder)
    
    # Check if it is a directory
    if os.path.isdir(class_path):
        # Count the number of image files in the class folder
        num_images = len([img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'))])
        class_counts[class_folder] = num_images

# Print the counts for each class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} images")


Class '.ipynb_checkpoints': 0 images
Class 'ADI': 1000 images
Class 'DEBRIS_MUCUS': 1000 images
Class 'LYM': 1000 images
Class 'MUSC_STROMA': 1000 images
Class 'NORM': 1000 images
Class 'TUM': 1000 images
